# Read in Data

In [39]:
import pandas as pd

data = pd.read_csv('data/player_visualization.csv', sep=',')
data = data.dropna()
data.head()

,Unnamed: 0,player,player_id,yearID,pos,WAR,out_of_contract
0,0,CJ Abrams,abramcj01,2023.0,h,3.4,False
1,1,CJ Abrams,abramcj01,2022.0,h,0.1,True
2,2,Jose Abreu,abreujo02,2023.0,h,0.0,True
3,3,Jose Abreu,abreujo02,2022.0,h,4.3,True
4,4,Jose Abreu,abreujo02,2021.0,h,2.9,True


# Run T Test

In [43]:
import warnings
from scipy import stats

# Load the data
data = pd.read_csv('data/player_visualization.csv', sep=',')

data.dropna()

# Group the data by 'player_id'
grouped = data.groupby('player_id')

# Initialize a dictionary to store the t-test results for each player
t_test_results = {}

# Perform the t-test for each player
for player_id, group in grouped:
    group1 = group[group['out_of_contract'] == True]['WAR']
    group2 = group[group['out_of_contract'] == False]['WAR']
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy='omit')
    t_test_results[player_id] = (t_statistic, p_value)

# Print the t-test results
for player_id, (t_statistic, p_value) in t_test_results.items():

    if p_value < 0.01:
        print(f"Player ID: {player_id}, t-statistic: {t_statistic}, p-value: {p_value}")

/Users/cadenparry/anaconda3/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Player ID: acostma01, t-statistic: inf, p-value: 0.0
Player ID: anderdr02, t-statistic: inf, p-value: 0.0
Player ID: andinro01, t-statistic: -7.5092822818395595, p-value: 6.8671525476252e-05
Player ID: andrima01, t-statistic: inf, p-value: 0.0
Player ID: bauertr01, t-statistic: 5.2915026221291805, p-value: 0.0011337831033596452
Player ID: bautijo02, t-statistic: -3.76513006726412, p-value: 0.002358595890423518
Player ID: colonba01, t-statistic: 2.920251505000728, p-value: 0.009136597867850814
Player ID: cruzne02, t-statistic: -3.9670719060482003, p-value: 0.000995791264099334
Player ID: delarru01, t-statistic: 5.855400437691198, p-value: 0.0020585239624297982
Player ID: derosma01, t-statistic: -4.269851563504603, p-value: 0.000777635677082984
Player ID: dicksbr01, t-statistic: -inf, p-value: 0.0
Player ID: drurybr01, t-statistic: -4.023387267936602, p-value: 0.0050378935391336395
Player ID: edmonji01, t-statistic: -4.172289123856086, p-value: 0.0008174213879330747
Player ID: garlajo01,